In [191]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [250]:
def data_split(data, n_columns):
    n_rows = int(len(data) / n_columns)
    return [data[i * n_columns:(i+1) * n_columns] for i in range(n_rows)]

In [207]:
def soup_to_dataframe(driver):
    html = driver.execute_script('return document.body.innerHTML;')
    soup = BeautifulSoup(html,'html.parser')
    
    # quarterly dates
    headers = [entry.text for entry
               in soup.find_all('div', {'role':'columnheader', 'class':'jqx-grid-column-header jqx-widget-header'})][2:]
    
    # values
    data = data_split([entry.text for entry in 
                       soup.find_all('div', {'role':'gridcell', 'class':'jqx-grid-cell jqx-item'})], len(headers))

    # labels
    labels = [entry.text for entry
              in soup.find_all('div', {'role':'gridcell', 'class':'jqx-grid-cell jqx-item jqx-grid-cell-pinned'})][::2]
    
    return pd.DataFrame(data, columns=headers, index=labels)

In [265]:
def scroll_bar_horizontal(scroll_bar, web_driver, x_offset):
    action = webdriver.common.action_chains.ActionChains(web_driver)
    action.move_to_element(scroll_bar).click_and_hold().move_by_offset(x_offset, 0).release().perform()
    time.sleep(0.5)

In [262]:
# getting data from macrotrends website


def get_info():
    print("There are four types of data you can get:\n1: financial-ratios\n2: cash-flow-statement\n"
    "3: balance-sheet\n4: income-statement\n\nThere is one function for getting the data:\n"
    "get_data(symbol, stock_name, financials_type)\n\nFor example: get_data('AAPL','apple','income-statement')\n\n"
    "The function returns pandas dataframe with the requested data.")

    
def get_data(symbol, stock_name, financials_type):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    link = f'https://www.macrotrends.net/stocks/charts/{symbol}/{stock_name}/{financials_type}?freq=Q'
    driver.get(link)
    driver.maximize_window()
    
    # create pandas dataframe from soup
    dataframe_1 = soup_to_dataframe(driver)
    
    # scrolling the bar to access hidden table data
    scrolling_bar = driver.find_element_by_id("jqxScrollThumbhorizontalScrollBarjqxgrid")
    scroll_bar_horizontal(scrolling_bar, driver, 200)
    
    dataframe_2 = soup_to_dataframe(driver)
    
    # concat dataframes together
    dataframe = pd.concat([dataframe_1, dataframe_2], axis=1)
    
    
    # remove dollar signs
    for i in range(len(dataframe)):
        for j in range(len(dataframe.columns)):
            if str(dataframe.iat[i, j]).startswith('$'):
                dataframe.iat[i, j] = dataframe.iat[i, j][1:]
    
    return dataframe